In [ ]:
#!pip install google-cloud-aiplatform vertexai tiktoken chromadb langchain transformers pytorch-transformers -U -q

In [ ]:
#import IPython
#IPython.Application.instance().kernel.do_shutdown(True)

## Imports

In [2]:
from datetime import datetime
import time
import pandas as pd
import numpy as np
import json
import os
import matplotlib.pyplot as plt
from IPython.display import Markdown

from langchain.llms import VertexAI
from langchain.embeddings import VertexAIEmbeddings
from langchain.schema import HumanMessage, SystemMessage
from langchain.docstore.document import Document

import vertexai
from vertexai.language_models import TextGenerationModel, TextEmbeddingModel
from vertexai.preview.language_models import TextGenerationModel as TextGenerationModel_preview
from google.cloud import aiplatform
from google.cloud import storage
from google.cloud import bigquery

print("Vertex AI version: " + str(aiplatform.__version__))

Vertex AI version: 1.33.0


## Env variables

In [3]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'mg-ce-demos'

In [4]:
REGION = 'us-central1'
LOCATION = 'us'
GCS_BUCKET = PROJECT_ID

In [11]:
palm_model = "text-bison"
palm_model_32k = "text-bison-32k"

## Summarization

### Doc loader

In [12]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

In [19]:
len(docs[0].page_content)
#display(Markdown(docs[0].page_content))

43519

### Quick start with load_summarize_chain

In [21]:
from langchain.chains.summarize import load_summarize_chain

llm = VertexAI(
    model_name=palm_model_32k,
    max_output_tokens=1024,
    temperature=0.4, 
    top_p=0.8,
    top_k=40,
    verbose=True,)
chain = load_summarize_chain(llm, chain_type="stuff")

response_lsc = chain.run(docs)
display(Markdown(response_lsc))

 Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos serve as inspiring examples. The potential of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver. 

An LLM-powered autonomous agent system typically involves a planning component, a memory component, and a tool use component. 

The planning component involves breaking down large tasks into smaller, manageable subgoals and reflecting on past actions to improve the quality of final results. 

The memory component includes short-term memory (in-context learning) and long-term memory (external vector store and fast retrieval). 

The tool use component enables the agent to learn to call external APIs for extra information that is missing from the model weights. 

Challenges include finite context length, challenges in long-term planning and task decomposition, and the reliability of the natural language interface.

### Stuffing

In [22]:
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain

# Define prompt
prompt_template = """Write a concise summary of the following:
"{text}"
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

# Define LLM chain
llm = VertexAI(
    model_name=palm_model_32k,
    max_output_tokens=1024,
    temperature=0.4, 
    top_p=0.8,
    top_k=40,
    verbose=True,)
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain, document_variable_name="text"
)

docs = loader.load()
response_stuff = stuff_chain.run(docs)
display(Markdown(response_stuff))

 Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos serve as inspiring examples. The potential of LLM extends beyond generating text; it can be framed as a powerful general problem solver. 

An LLM-powered autonomous agent system typically involves a planning component, a memory component, and a tool use component. 

The planning component involves breaking down large tasks into smaller, manageable subgoals and reflecting on past actions to improve future steps. 

The memory component includes short-term memory (in-context learning) and long-term memory (external vector store with fast retrieval). 

The tool use component enables the agent to call external APIs for missing information, including current information, code execution capability, and access to proprietary information sources. 

Challenges include finite context length, challenges in long-term planning and task decomposition, and the reliability of the natural language interface. 

Despite these challenges, LLM-powered autonomous agents have the potential to be powerful tools in various domains.


### Map-reduce

In [23]:
from langchain.chains.mapreduce import MapReduceChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ReduceDocumentsChain, MapReduceDocumentsChain

llm = VertexAI(
    model_name=palm_model_32k,
    max_output_tokens=1024,
    temperature=0.4, 
    top_p=0.8,
    top_k=40,
    verbose=True,)

# Map
map_template = """The following is a set of documents
{docs}
Based on this list of docs, please identify the main themes 
Helpful Answer:"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

#from langchain import hub
#map_prompt = hub.pull("rlm/map-prompt")
#map_chain = LLMChain(llm=llm, prompt=map_prompt)

In [24]:
# Reduce
reduce_template = """The following is set of summaries:
{doc_summaries}
Take these and distill it into a final, consolidated summary of the main themes. 
Helpful Answer:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)

# Note we can also get this from the prompt hub, as noted above
#reduce_prompt = hub.pull("rlm/map-prompt")

In [25]:
# Run chain
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="doc_summaries"
)

# Combines and iteravely reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=4000,
)

In [28]:
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)

Created a chunk of size 1003, which is longer than the specified 1000


In [29]:
response_map_reduce = map_reduce_chain.run(split_docs)
print(response_map_reduce)
#display(Markdown(response_map_reduce))

ImportError: Could not import transformers python package. This is needed in order to calculate get_token_ids. Please install it with `pip install transformers`.